In [1]:
#!pip install haversine

In [7]:
from haversine import haversine, Unit
import pandas as pd
import numpy as np
import time

In [8]:
df_standard = pd.read_csv('서울시 카페 유동인구.csv')
#df_standard

In [9]:
df_bus = pd.read_csv('서울시_버스정류장.csv',encoding='cp949')
df_parking = pd.read_csv('서울시_공영주차장.csv')
df_cafe = pd.read_csv('서울시 카페.csv')
df_bicycle = pd.read_csv('서울시_자전거대여소.csv',encoding='cp949')
df_subway = pd.read_csv('지하철역_좌표.csv')
df_school = pd.read_csv('서울시학교.csv')
df_dongcode = pd.read_csv('행정동코드변환.csv')
df_store = pd.read_csv('서울시 소모품점.csv')
df_restaurant = pd.read_csv('서울시 음식점.csv')

In [10]:
def return_N(standard_lat, standard_long, compare_lat, compare_long):
  result = []
  for i in range(len(standard_lat)):
    z = []
    standard = (standard_lat[i], standard_long[i])
    for j in range(len(compare_lat)):
      compare = (compare_lat[j], compare_long[j])
      x = haversine(standard, compare, unit='m') <= 500
      z.append(x)
    result.append(sum(z))
  return result

In [11]:
standard_lat = np.array(df_standard.latitude.values)
standard_long = np.array(df_standard.longitude.values)

In [12]:
%%time
#df_bus

compare_lat = np.array(df_bus.Y좌표.values)
compare_long = np.array(df_bus.X좌표.values)

df_standard['bus_N'] = return_N(standard_lat, standard_long, compare_lat, compare_long)

Wall time: 11min 26s


In [13]:
df_parking = df_parking.drop(df_parking.loc[df_parking["longitude"] >=180].index)
df_parking = df_parking.drop(df_parking.loc[df_parking["latitude"] >=90].index)

In [14]:
%%time
#df_parking

compare_lat = np.array(df_parking.latitude.values)
compare_long = np.array(df_parking.longitude.values)

df_standard['parking_N'] = return_N(standard_lat, standard_long, compare_lat, compare_long)

Wall time: 15min 29s


In [15]:
%%time
#df_cafe

compare_lat = np.array(df_cafe.latitude.values)
compare_long = np.array(df_cafe.longitude.values)

df_standard['cafe_N'] = return_N(standard_lat, standard_long, compare_lat, compare_long)

Wall time: 7min 25s


In [16]:
%%time
#df_bicycle

compare_lat = np.array(df_bicycle.y.values)
compare_long = np.array(df_bicycle.x.values)

df_standard['bicycle_N'] = return_N(standard_lat, standard_long, compare_lat, compare_long)

Wall time: 2min 50s


In [17]:
%%time
#df_subway

df_compare = df_subway.T
df_compare.columns=['latitude','longitude']
df_compare = df_compare.drop('Unnamed: 0', axis=0)

compare_lat = np.array(df_compare.latitude.values)
compare_long = np.array(df_compare.longitude.values)

df_standard['subway_N'] = return_N(standard_lat, standard_long, compare_lat, compare_long)

Wall time: 37.9 s


In [18]:
%%time
#df_school

compare_lat = np.array(df_school.위도.values)
compare_long = np.array(df_school.경도.values)

df_standard['school_N'] = return_N(standard_lat, standard_long, compare_lat, compare_long)

Wall time: 2min 38s


In [19]:
def return_s(standard_lat, standard_long, compare_lat, compare_long):
  result = []
  for i in range(len(standard_lat)):
    z = {}
    standard = (standard_lat[i], standard_long[i])
    for j in range(len(compare_lat)):
      compare = (compare_lat[j], compare_long[j])
      x = haversine(standard, compare, unit='m')
      z[area_code[j]] = x
    all_values = z.values()
    min_value = min(all_values)
    for key, value in z.items():
        if value == min_value:
            ca_code = key
    result.append(ca_code)
  return result

In [20]:
def return_d(standard_lat, standard_long, compare_lat, compare_long):
  result = []
  for i in range(len(standard_lat)):
    z = []
    standard = (standard_lat[i], standard_long[i])
    for j in range(len(compare_lat)):
      compare = (compare_lat[j], compare_long[j])
      x = haversine(standard, compare, unit='m')
      z.append(x)
    result.append(min(z))
  return result

In [21]:
%%time
#df_dongcode

compare_lat = np.array(df_dongcode.latitude.values)
compare_long = np.array(df_dongcode.longitude.values)
area_code = np.array(df_dongcode.상권_코드.values)


df_standard['dongcode'] = return_s(standard_lat, standard_long, compare_lat, compare_long)

Wall time: 1min 46s


In [22]:
df_sales = pd.read_csv("서울시_우리마을가게_상권분석서비스(신_상권_추정매출)_2021년.csv",encoding='cp949')
df_sales_coffee = df_sales.loc[df_sales['서비스_업종_코드_명'] == '커피-음료']
df_sales_coffee = df_sales_coffee[["상권_코드",'분기당_매출_금액','점포수']]
df_sales_coffee['평균매출'] = df_sales_coffee['분기당_매출_금액']/df_sales_coffee['점포수']
sales_dict = list(zip(df_sales_coffee["상권_코드"],df_sales_coffee["평균매출"]))
sales_dict = dict(sales_dict)

sales_list = []
for i in df_standard["dongcode"]:
    sales = sales_dict.get(i)
    sales_list.append(sales)
df_standard['aver_sales'] = sales_list    

In [23]:
%%time
#df_subway

df_compare = df_subway.T
df_compare.columns=['latitude','longitude']
df_compare = df_compare.drop('Unnamed: 0', axis=0)

compare_lat = np.array(df_compare.latitude.values)
compare_long = np.array(df_compare.longitude.values)

df_standard['subway_d'] = return_d(standard_lat, standard_long, compare_lat, compare_long)

Wall time: 37.2 s


In [24]:
%%time
#df_store

compare_lat = np.array(df_store.latitude.values)
compare_long = np.array(df_store.longitude.values)

df_standard['store_N'] = return_N(standard_lat, standard_long, compare_lat, compare_long)

Wall time: 14min 18s


In [25]:
%%time
#df_restauran

compare_lat = np.array(df_restaurant.latitude.values)
compare_long = np.array(df_restaurant.longitude.values)

df_standard['rest_N'] = return_N(standard_lat, standard_long, compare_lat, compare_long)

Wall time: 7min 36s


In [26]:
df_standard

,name,latitude,longitude,address,review,savecount,visitorreviewscore,flow_pop,bus_N,parking_N,cafe_N,bicycle_N,subway_N,school_N,dongcode,aver_sales,subway_d,store_N,rest_N
0,칼리프하우스,37.524966,127.036492,서울 강남구 도산대로49길 29-8,23,"7,000",NaN,8434,17,91,67,5,1,0,2110962,7.453085e+07,446.329032,115,68
1,런던 베이글 뮤지엄 도산점,37.526075,127.036449,"서울 강남구 언주로168길 33 1, 2층",310,"23,000",NaN,5844,14,76,66,5,1,2,2110962,7.453085e+07,388.399142,111,66
2,꽁티드툴레아,37.525570,127.036905,서울 강남구 도산대로49길 39,"2,813","91,000",NaN,5844,15,71,65,5,1,0,2110962,7.453085e+07,378.148659,111,67
3,썸띵어바웃커피,37.502845,127.027839,"서울 강남구 강남대로102길 30 1, 2, 3층","1,977","38,000",4.42,494,32,1,58,4,1,2,2110954,1.282070e+08,313.215103,132,88
4,올드페리도넛 가로수길점,37.520415,127.021299,"서울 강남구 강남대로160길 35-5 지하1층 1층, 2층",91,"1,000",NaN,2052,12,1,59,5,1,4,2120186,3.363400e+07,465.658687,103,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7349,후식 상봉역점,37.596717,127.085720,서울 중랑구 망우로 297 상봉역사 내,26,~100,4.61,5385,16,83,35,8,1,2,2120069,1.995080e+07,72.366733,86,60
7350,디저트39 중랑신내SKV1점,37.613155,127.112358,서울 중랑구 신내역로 111 B동 133호 (1층) CU 옆,108,~100,NaN,275,6,1,1,2,0,0,2110279,NaN,805.590656,2,0
7351,로플로플 사가정역점,37.582736,127.088254,서울 중랑구 면목로49길 6,37,~100,4.46,969,25,1,44,3,1,2,2120068,2.346771e+07,205.784727,79,43
7352,작은창고21,37.588509,127.079169,서울 중랑구 동일로101길 11-21 1층,40,~100,4.45,999,28,12,23,4,0,4,2110243,2.742376e+06,734.360132,30,7


In [27]:
df_standard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7354 entries, 0 to 7353
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                7354 non-null   object 
 1   latitude            7354 non-null   float64
 2   longitude           7354 non-null   float64
 3   address             7353 non-null   object 
 4   review              7324 non-null   object 
 5   savecount           7354 non-null   object 
 6   visitorreviewscore  5519 non-null   float64
 7   flow_pop            7354 non-null   int64  
 8   bus_N               7354 non-null   int64  
 9   parking_N           7354 non-null   int64  
 10  cafe_N              7354 non-null   int64  
 11  bicycle_N           7354 non-null   int64  
 12  subway_N            7354 non-null   int64  
 13  school_N            7354 non-null   int64  
 14  dongcode            7354 non-null   int64  
 15  aver_sales          6245 non-null   float64
 16  subway

In [28]:
df_standard

,name,latitude,longitude,address,review,savecount,visitorreviewscore,flow_pop,bus_N,parking_N,cafe_N,bicycle_N,subway_N,school_N,dongcode,aver_sales,subway_d,store_N,rest_N
0,칼리프하우스,37.524966,127.036492,서울 강남구 도산대로49길 29-8,23,"7,000",NaN,8434,17,91,67,5,1,0,2110962,7.453085e+07,446.329032,115,68
1,런던 베이글 뮤지엄 도산점,37.526075,127.036449,"서울 강남구 언주로168길 33 1, 2층",310,"23,000",NaN,5844,14,76,66,5,1,2,2110962,7.453085e+07,388.399142,111,66
2,꽁티드툴레아,37.525570,127.036905,서울 강남구 도산대로49길 39,"2,813","91,000",NaN,5844,15,71,65,5,1,0,2110962,7.453085e+07,378.148659,111,67
3,썸띵어바웃커피,37.502845,127.027839,"서울 강남구 강남대로102길 30 1, 2, 3층","1,977","38,000",4.42,494,32,1,58,4,1,2,2110954,1.282070e+08,313.215103,132,88
4,올드페리도넛 가로수길점,37.520415,127.021299,"서울 강남구 강남대로160길 35-5 지하1층 1층, 2층",91,"1,000",NaN,2052,12,1,59,5,1,4,2120186,3.363400e+07,465.658687,103,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7349,후식 상봉역점,37.596717,127.085720,서울 중랑구 망우로 297 상봉역사 내,26,~100,4.61,5385,16,83,35,8,1,2,2120069,1.995080e+07,72.366733,86,60
7350,디저트39 중랑신내SKV1점,37.613155,127.112358,서울 중랑구 신내역로 111 B동 133호 (1층) CU 옆,108,~100,NaN,275,6,1,1,2,0,0,2110279,NaN,805.590656,2,0
7351,로플로플 사가정역점,37.582736,127.088254,서울 중랑구 면목로49길 6,37,~100,4.46,969,25,1,44,3,1,2,2120068,2.346771e+07,205.784727,79,43
7352,작은창고21,37.588509,127.079169,서울 중랑구 동일로101길 11-21 1층,40,~100,4.45,999,28,12,23,4,0,4,2110243,2.742376e+06,734.360132,30,7


In [30]:
df_standard.to_csv('cafe_df_pipe.csv',index=False)